In [1]:
import cv2
import dlib
import numpy as np
import os
import pandas as pd # Import pandas for CSV handling

# Load detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(
    "models/shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat"
)

# Define the root directories for input and output
root_input_dir = "archive_3"
root_output_dir = "processed_faces_newFinal8_1"

# Define the scaling factor for upscaling images
scale_factor = 12

# Create the root output directory if it doesn't exist
os.makedirs(root_output_dir, exist_ok=True)

# List of emotion folders
emotion_folders = ["anger", "contempt", "disgust", "happy", "sadness", "fear", "surprise"]

# List to store all landmark and fitted shape data for CSV export
all_facial_features_data = [] # Renamed for clarity to include more than just raw landmarks

print("Starting facial landmark detection, shape drawing, and feature extraction...")
print("Output images will be split into '_with_photo', '_without_photo', and '_shapes_drawn' folders.")
print("A comprehensive CSV file with landmark coordinates AND fitted shape parameters will be generated.")

# --- Helper Function for Drawing Shapes (from previous correction) ---
def draw_facial_shapes(image, landmarks):
    """
    Draws different geometric shapes on a given image based on facial landmarks.
    Args:
        image (numpy.ndarray): The image on which to draw.
        landmarks (dlib.full_object_detection): Dlib's landmark object.
    Returns:
        numpy.ndarray: The image with shapes drawn.
    """
    drawn_image = image.copy()

    # Define colors for different shapes (BGR format)
    color_eyebrows = (0, 255, 0)   # Green
    color_eyes = (255, 0, 0)       # Blue
    color_nose_bridge = (0, 255, 255) # Yellow
    color_nose_tip = (255, 255, 0) # Cyan
    color_lips = (0, 165, 255)     # Orange
    color_cheeks = (128, 0, 128)   # Purple
    thickness = 2 # Line thickness

    # --- Eyebrows (Approximated by polylines) ---
    # Left Eyebrow: 17-21
    for i in range(17, 21):
        pt1 = (landmarks.part(i).x, landmarks.part(i).y)
        pt2 = (landmarks.part(i + 1).x, landmarks.part(i + 1).y)
        cv2.line(drawn_image, pt1, pt2, color_eyebrows, thickness)

    # Right Eyebrow: 22-26
    for i in range(22, 26):
        pt1 = (landmarks.part(i).x, landmarks.part(i).y)
        pt2 = (landmarks.part(i + 1).x, landmarks.part(i + 1).y)
        cv2.line(drawn_image, pt1, pt2, color_eyebrows, thickness)

    # --- Eyes (Ellipses) ---
    # Left Eye: 36-41
    left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
    if len(left_eye_points) >= 5: # cv2.fitEllipse requires at least 5 points
        try: # Add try-except for robustness against poor point configurations
            (center_le, axes_le, angle_le) = cv2.fitEllipse(left_eye_points)
            if axes_le[0] > 0 and axes_le[1] > 0: # Ensure axes are valid
                cv2.ellipse(drawn_image, (int(center_le[0]), int(center_le[1])),
                            (int(axes_le[0] / 2), int(axes_le[1] / 2)),
                            angle_le, 0, 360, color_eyes, thickness)
        except cv2.error:
            # print(f"Warning: Could not fit ellipse for left eye. Skipping drawing.")
            pass # Suppress drawing error, fitting will use NaN

    # Right Eye: 42-47
    right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])
    if len(right_eye_points) >= 5:
        try:
            (center_re, axes_re, angle_re) = cv2.fitEllipse(right_eye_points)
            if axes_re[0] > 0 and axes_re[1] > 0:
                cv2.ellipse(drawn_image, (int(center_re[0]), int(center_re[1])),
                            (int(axes_re[0] / 2), int(axes_re[1] / 2)),
                            angle_re, 0, 360, color_eyes, thickness)
        except cv2.error:
            # print(f"Warning: Could not fit ellipse for right eye. Skipping drawing.")
            pass # Suppress drawing error, fitting will use NaN


    # --- Nose ---
    # Nose Bridge (Trapezium): Using landmarks 27, 28, 29, 30
    nose_bridge_pts = np.array([
        (landmarks.part(27).x, landmarks.part(27).y),
        (landmarks.part(28).x, landmarks.part(28).y),
        (landmarks.part(29).x, landmarks.part(29).y),
        (landmarks.part(30).x, landmarks.part(30).y)
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [nose_bridge_pts], True, color_nose_bridge, thickness)

    # Nose Tip (Triangle): Using landmarks 30, 31, 35
    nose_tip_pts = np.array([
        (landmarks.part(30).x, landmarks.part(30).y),
        (landmarks.part(31).x, landmarks.part(31).y),
        (landmarks.part(35).x, landmarks.part(35).y)
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [nose_tip_pts], True, color_nose_tip, thickness)

    # --- Lips (Outer Lip Ellipse) ---
    # Outer Lip: 48-59
    outer_lip_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 60)])
    if len(outer_lip_points) >= 5:
        try:
            (center_ol, axes_ol, angle_ol) = cv2.fitEllipse(outer_lip_points)
            if axes_ol[0] > 0 and axes_ol[1] > 0:
                cv2.ellipse(drawn_image, (int(center_ol[0]), int(center_ol[1])),
                            (int(axes_ol[0] / 2), int(axes_ol[1] / 2)),
                            angle_ol, 0, 360, color_lips, thickness)
        except cv2.error:
            # print(f"Warning: Could not fit ellipse for outer lip. Skipping drawing.")
            pass # Suppress drawing error, fitting will use NaN

    # --- Cheeks (Approximated by Broader Triangles) ---
    # Left Cheek: Using outer eye corner (36), outer jawline (2), left mouth corner (48)
    left_cheek_pts = np.array([
        (landmarks.part(36).x, landmarks.part(36).y), # Left outer eye corner
        (landmarks.part(2).x, landmarks.part(2).y),   # Left jawline (near chin)
        (landmarks.part(48).x, landmarks.part(48).y)  # Left mouth corner
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [left_cheek_pts], True, color_cheeks, thickness)

    # Right Cheek: Using outer eye corner (45), outer jawline (14), right mouth corner (54)
    right_cheek_pts = np.array([
        (landmarks.part(45).x, landmarks.part(45).y), # Right outer eye corner
        (landmarks.part(14).x, landmarks.part(14).y), # Right jawline (near chin)
        (landmarks.part(54).x, landmarks.part(54).y)  # Right mouth corner
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [right_cheek_pts], True, color_cheeks, thickness)

    return drawn_image

# --- Helper Functions for Curve Fitting ---

def fit_parabola_params(points):
    """
    Fits a 2nd-degree polynomial (parabola) to a set of (x, y) points.
    Returns (a, b, c) for y = ax^2 + bx + c.
    Returns (NaN, NaN, NaN) if fitting fails.
    """
    if len(points) < 3: # Need at least 3 points to fit a parabola
        return np.nan, np.nan, np.nan
    x_coords = np.array([p[0] for p in points])
    y_coords = np.array([p[1] for p in points])
    try:
        # polyfit can sometimes raise warnings for ill-conditioned matrices
        # Suppress warnings if you are sure about the input data
        with np.errstate(all='raise'):
            coefficients = np.polyfit(x_coords, y_coords, 2)
            return tuple(coefficients) # (a, b, c)
    except Exception as e: # Catch numpy polynomial fitting errors
        # print(f"Warning: Parabola fitting failed: {e}")
        return np.nan, np.nan, np.nan

def fit_ellipse_params(points):
    """
    Fits an ellipse to a set of (x, y) points using cv2.fitEllipse.
    Returns (center_x, center_y, major_axis, minor_axis, angle)
    Returns (NaN, NaN, NaN, NaN, NaN) if fitting fails or points are insufficient.
    """
    if len(points) < 5: # cv2.fitEllipse requires at least 5 points
        return np.nan, np.nan, np.nan, np.nan, np.nan
    points_np = np.array(points, dtype=np.int32)
    try:
        (center, axes, angle) = cv2.fitEllipse(points_np)
        # Ensure axes are positive; sometimes very small or negative can be returned
        if axes[0] <= 0 or axes[1] <= 0:
            return np.nan, np.nan, np.nan, np.nan, np.nan

        # axes are (major_axis_length, minor_axis_length) or vice-versa.
        # It's diameter, so convert to radius if needed for other applications.
        # For now, we'll store diameter values as returned.
        return center[0], center[1], axes[0], axes[1], angle
    except cv2.error as e:
        # print(f"Warning: Ellipse fitting failed: {e}")
        return np.nan, np.nan, np.nan, np.nan, np.nan
    except Exception as e: # Catch any other unexpected errors
        # print(f"Warning: Ellipse fitting failed unexpectedly: {e}")
        return np.nan, np.nan, np.nan, np.nan, np.nan


# --- Main Processing Loop ---
for emotion in emotion_folders:
    input_emotion_path = os.path.join(root_input_dir, emotion)

    # Define the new output subfolders for each emotion
    output_with_photo_path = os.path.join(root_output_dir, emotion, f"{emotion}_with_photo")
    output_without_photo_path = os.path.join(root_output_dir, emotion, f"{emotion}_without_photo")
    output_shapes_drawn_path = os.path.join(root_output_dir, emotion, f"{emotion}_shapes_drawn")

    # Create these new output subdirectories if they don't exist
    os.makedirs(output_with_photo_path, exist_ok=True)
    os.makedirs(output_without_photo_path, exist_ok=True)
    os.makedirs(output_shapes_drawn_path, exist_ok=True)

    print(f"\nProcessing emotion: {emotion}...")

    for filename in os.listdir(input_emotion_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            image_path = os.path.join(input_emotion_path, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Warning: Could not read image {image_path}. Skipping.")
                continue

            print(f"  Processing file: {filename}")

            upscaled_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
            gray = cv2.cvtColor(upscaled_image, cv2.COLOR_BGR2GRAY)

            faces = detector(gray)

            if len(faces) > 0:
                face = faces[0]
                landmarks = predictor(gray, face)

                # --- Initialize row for CSV data ---
                current_image_features = {'emotion': emotion, 'filename': filename}

                # --- Collect raw landmark data for CSV ---
                for n in range(68):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y
                    current_image_features[f'landmark_{n}_x'] = x
                    current_image_features[f'landmark_{n}_y'] = y

                # --- Apply Curve Fitting Algorithms and Store Parameters ---

                # Left Eyebrow (Parabola: y = ax^2 + bx + c)
                leb_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(17, 22)]
                a_leb, b_leb, c_leb = fit_parabola_params(leb_points)
                current_image_features['leb_parabola_a'] = a_leb
                current_image_features['leb_parabola_b'] = b_leb
                current_image_features['leb_parabola_c'] = c_leb

                # Right Eyebrow (Parabola: y = ax^2 + bx + c)
                reb_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(22, 27)]
                a_reb, b_reb, c_reb = fit_parabola_params(reb_points)
                current_image_features['reb_parabola_a'] = a_reb
                current_image_features['reb_parabola_b'] = b_reb
                current_image_features['reb_parabola_c'] = c_reb

                # Left Eye (Ellipse: center_x, center_y, major_axis, minor_axis, angle)
                le_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)]
                cx_le, cy_le, maj_le, min_le, ang_le = fit_ellipse_params(le_points)
                current_image_features['le_ellipse_center_x'] = cx_le
                current_image_features['le_ellipse_center_y'] = cy_le
                current_image_features['le_ellipse_major_axis'] = maj_le
                current_image_features['le_ellipse_minor_axis'] = min_le
                current_image_features['le_ellipse_angle'] = ang_le

                # Right Eye (Ellipse)
                re_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)]
                cx_re, cy_re, maj_re, min_re, ang_re = fit_ellipse_params(re_points)
                current_image_features['re_ellipse_center_x'] = cx_re
                current_image_features['re_ellipse_center_y'] = cy_re
                current_image_features['re_ellipse_major_axis'] = maj_re
                current_image_features['re_ellipse_minor_axis'] = min_re
                current_image_features['re_ellipse_angle'] = ang_re

                # Outer Lip (Ellipse)
                ol_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 60)]
                cx_ol, cy_ol, maj_ol, min_ol, ang_ol = fit_ellipse_params(ol_points)
                current_image_features['ol_ellipse_center_x'] = cx_ol
                current_image_features['ol_ellipse_center_y'] = cy_ol
                current_image_features['ol_ellipse_major_axis'] = maj_ol
                current_image_features['ol_ellipse_minor_axis'] = min_ol
                current_image_features['ol_ellipse_angle'] = ang_ol

                # Nose Bridge (Trapezium - extract vertices)
                current_image_features['nose_bridge_v1_x'] = landmarks.part(27).x
                current_image_features['nose_bridge_v1_y'] = landmarks.part(27).y
                current_image_features['nose_bridge_v2_x'] = landmarks.part(28).x
                current_image_features['nose_bridge_v2_y'] = landmarks.part(28).y
                current_image_features['nose_bridge_v3_x'] = landmarks.part(29).x
                current_image_features['nose_bridge_v3_y'] = landmarks.part(29).y
                current_image_features['nose_bridge_v4_x'] = landmarks.part(30).x
                current_image_features['nose_bridge_v4_y'] = landmarks.part(30).y

                # Nose Tip (Triangle - extract vertices)
                current_image_features['nose_tip_v1_x'] = landmarks.part(30).x
                current_image_features['nose_tip_v1_y'] = landmarks.part(30).y
                current_image_features['nose_tip_v2_x'] = landmarks.part(31).x
                current_image_features['nose_tip_v2_y'] = landmarks.part(31).y
                current_image_features['nose_tip_v3_x'] = landmarks.part(35).x
                current_image_features['nose_tip_v3_y'] = landmarks.part(35).y

                # Left Cheek (Triangle - extract vertices)
                current_image_features['left_cheek_v1_x'] = landmarks.part(36).x
                current_image_features['left_cheek_v1_y'] = landmarks.part(36).y
                current_image_features['left_cheek_v2_x'] = landmarks.part(2).x
                current_image_features['left_cheek_v2_y'] = landmarks.part(2).y
                current_image_features['left_cheek_v3_x'] = landmarks.part(48).x
                current_image_features['left_cheek_v3_y'] = landmarks.part(48).y

                # Right Cheek (Triangle - extract vertices)
                current_image_features['right_cheek_v1_x'] = landmarks.part(45).x
                current_image_features['right_cheek_v1_y'] = landmarks.part(45).y
                current_image_features['right_cheek_v2_x'] = landmarks.part(14).x
                current_image_features['right_cheek_v2_y'] = landmarks.part(14).y
                current_image_features['right_cheek_v3_x'] = landmarks.part(54).x
                current_image_features['right_cheek_v3_y'] = landmarks.part(54).y
                
                
                # --- Advanced Geometric Features (Distances, Ratios, Angles) ---

                def euclidean(pt1, pt2):
                    return np.linalg.norm(np.array(pt1) - np.array(pt2))

                def calculate_angle(pt1, pt2, pt3):
                    a = np.array(pt1)
                    b = np.array(pt2)
                    c = np.array(pt3)
                    ba = a - b
                    bc = c - b
                    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                    return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

                # Helper: Get (x, y) for a landmark
                def get_pt(idx): return (landmarks.part(idx).x, landmarks.part(idx).y)

                # Distance between eyes
                eye_left = get_pt(36)
                eye_right = get_pt(45)
                current_image_features['eye_distance'] = euclidean(eye_left, eye_right)

                # EAR for Left Eye (36-41)
                eye_l = [get_pt(i) for i in range(36, 42)]
                A = euclidean(eye_l[1], eye_l[5])
                B = euclidean(eye_l[2], eye_l[4])
                C = euclidean(eye_l[0], eye_l[3])
                current_image_features['left_eye_ear'] = (A + B) / (2.0 * C)

                # EAR for Right Eye (42-47)
                eye_r = [get_pt(i) for i in range(42, 48)]
                A = euclidean(eye_r[1], eye_r[5])
                B = euclidean(eye_r[2], eye_r[4])
                C = euclidean(eye_r[0], eye_r[3])
                current_image_features['right_eye_ear'] = (A + B) / (2.0 * C)

                # MAR (Mouth Aspect Ratio) using outer mouth points 60–67
                mouth = [get_pt(i) for i in range(60, 68)]
                A = euclidean(mouth[2], mouth[10 % 8])
                B = euclidean(mouth[3], mouth[9 % 8])
                C = euclidean(mouth[4], mouth[8 % 8])
                horizontal = euclidean(mouth[0], mouth[6])
                current_image_features['mouth_mar'] = (A + B + C) / (3.0 * horizontal)

                # Nose Angle: points 27 (nose top), 30 (tip), 8 (chin)
                current_image_features['nose_angle'] = calculate_angle(get_pt(27), get_pt(30), get_pt(8))

                # Eyebrow Angle (slope between start and end of left eyebrow: 17–21)
                current_image_features['left_eyebrow_angle'] = calculate_angle(get_pt(19), get_pt(17), get_pt(21))


                # Add the collected and fitted data row to the main list
                all_facial_features_data.append(current_image_features)

                # --- Prepare images for all output types (visualizations) ---
                drawn_image_with_photo = upscaled_image.copy()
                drawn_image_without_photo = np.zeros(upscaled_image.shape, dtype=np.uint8)
                drawn_image_shapes_only = np.zeros(upscaled_image.shape, dtype=np.uint8)

                # Get position of landmark 31 for quadrant lines (on _with_photo image)
                nose_x = landmarks.part(30).x
                nose_y = landmarks.part(30).y
                img_h, img_w = drawn_image_with_photo.shape[:2]
                cv2.line(drawn_image_with_photo, (nose_x, 0), (nose_x, img_h), (0, 255, 0), 1)
                cv2.line(drawn_image_with_photo, (0, nose_y), (img_w, nose_y), (0, 255, 0), 1)

                drawn_index = 1
                seen = set()

                for n in range(68):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y

                    if (x, y) in seen:
                        continue
                    seen.add((x, y))

                    circle_radius = 2
                    fill_type = -1

                    # Offset for label (re-using your existing logic)
                    offset_x, offset_y = -6, 10
                    if n in [37, 38, 39, 43, 44, 45]: # Upper eye points
                        offset_y = -10
                    elif n == 49: # Left mouth corner
                        offset_x = -20
                        offset_y = 6
                    elif n == 50: # Top lip center
                        offset_x = 0
                        offset_y = -8
                    elif n == 65: # Chin left
                        offset_x = -14
                        offset_y = 10
                    elif 48 <= n <= 54: # Upper lip
                        offset_y = -8
                    elif 55 <= n <= 59: # Lower lip center to right
                        offset_y = 10
                    elif 60 <= n <= 64: # Inner mouth top
                        offset_y = -8
                    elif 65 <= n <= 67: # Chin right
                        offset_y = 10

                    # Draw on the image WITH photo (original background)
                    text_color_with_photo = (0, 0, 0) # Black text
                    cv2.circle(drawn_image_with_photo, (x, y), circle_radius, (0, 0, 255), fill_type) # Red circles
                    cv2.putText(drawn_image_with_photo, str(drawn_index), (x + offset_x, y + offset_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, text_color_with_photo, 1, cv2.LINE_AA)

                    # Draw on the image WITHOUT photo (black background)
                    text_color_without_photo = (255, 255, 255) # White text/circles
                    cv2.circle(drawn_image_without_photo, (x, y), circle_radius, text_color_without_photo, fill_type)
                    cv2.putText(drawn_image_without_photo, str(drawn_index), (x + offset_x, y + offset_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, text_color_without_photo, 1, cv2.LINE_AA)

                    drawn_index += 1

                # --- Draw the specific shapes on the dedicated 'shapes_only' image ---
                drawn_image_shapes_only = draw_facial_shapes(drawn_image_shapes_only, landmarks)

                # Save the processed images to their respective folders
                output_image_path_with_photo = os.path.join(output_with_photo_path, filename)
                cv2.imwrite(output_image_path_with_photo, drawn_image_with_photo)

                output_image_path_without_photo = os.path.join(output_without_photo_path, filename)
                cv2.imwrite(output_image_path_without_photo, drawn_image_without_photo)

                output_image_path_shapes_drawn = os.path.join(output_shapes_drawn_path, filename)
                cv2.imwrite(output_image_path_shapes_drawn, drawn_image_shapes_only)

            else:
                print(f"  No face detected in {filename} from {emotion} folder.")

print("\nAll emotion folders and photos processed successfully!")

# --- Save all collected facial feature data to a single CSV file ---
if all_facial_features_data:
    df = pd.DataFrame(all_facial_features_data)
    # Ensure all columns are present, even if some fits failed for certain images (filled with NaN)
    # CORRECTED LINE: Removed 'axis=1'
    df = df.reindex(columns=sorted(df.columns)) # Optional: sort columns alphabetically
    csv_output_path = os.path.join(root_output_dir, "facial_features_and_landmarks_data.csv")
    df.to_csv(csv_output_path, index=False)
    print(f"All facial landmark coordinates and fitted shape parameters saved to: {csv_output_path}")
else:
    print("No facial feature data was collected (no faces detected or no images processed).")

Starting facial landmark detection, shape drawing, and feature extraction...
Output images will be split into '_with_photo', '_without_photo', and '_shapes_drawn' folders.
A comprehensive CSV file with landmark coordinates AND fitted shape parameters will be generated.

Processing emotion: anger...
  Processing file: S010_004_00000017.png
  Processing file: S010_004_00000018.png
  Processing file: S010_004_00000019.png
  Processing file: S011_004_00000019.png
  Processing file: S011_004_00000020.png
  Processing file: S011_004_00000021.png
  Processing file: S014_003_00000028.png
  Processing file: S014_003_00000029.png
  Processing file: S014_003_00000030.png
  Processing file: S022_005_00000030.png
  Processing file: S022_005_00000031.png
  Processing file: S022_005_00000032.png
  Processing file: S026_003_00000013.png
  Processing file: S026_003_00000014.png
  Processing file: S026_003_00000015.png
  Processing file: S028_001_00000022.png
  Processing file: S028_001_00000023.png
  P

In [2]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

# Step 1: Define paths
input_csv_path = "processed_faces_newFinal8_1/facial_features_and_landmarks_data.csv"
output_folder = "processed_faces_newFinal8_1"
output_csv_path = os.path.join(output_folder, "facial_features_and_landmarks_data_cleaned.csv")

# Step 2: Create the new output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Step 3: Load the original dataset
df = pd.read_csv(input_csv_path)
print(f"Original dataset shape: {df.shape}")

# Step 4: Drop non-feature columns
if 'filename' in df.columns:
    df = df.drop(columns=["filename"])

# Step 5: Encode 'emotion' column (target variable)
if 'emotion' in df.columns:
    le = LabelEncoder()
    df['emotion'] = le.fit_transform(df['emotion'])  # e.g., anger → 0, happy → 1, etc.
    print(f"Encoded emotion labels: {dict(zip(le.classes_, le.transform(le.classes_)))}")

# Step 6: Drop rows with any missing values (NaNs from failed fittings)
df_cleaned = df.dropna()
print(f"Cleaned dataset shape after dropping NaNs: {df_cleaned.shape}")

# Step 7: Save the cleaned dataset
df_cleaned.to_csv(output_csv_path, index=False)
print(f"Cleaned data saved to: {output_csv_path}")


Original dataset shape: (980, 191)
Encoded emotion labels: {'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happy': 4, 'sadness': 5, 'surprise': 6}
Cleaned dataset shape after dropping NaNs: (980, 190)
Cleaned data saved to: processed_faces_newFinal8_1\facial_features_and_landmarks_data_cleaned.csv


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os

# Step 1: Load the cleaned dataset
input_csv_path = "processed_faces_newFinal8_1/facial_features_and_landmarks_data_cleaned.csv"
df = pd.read_csv(input_csv_path)
print(f"Dataset shape: {df.shape}")

# Step 2: Split features (X) and labels (y)
X = df.drop(columns=["emotion"])
y = df["emotion"]

# Step 3: Train-test split (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 4: Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Train Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

# Step 6: Predict on test set
y_pred = model.predict(X_test_scaled)

# Step 7: Accuracy Scores
train_accuracy = model.score(X_train_scaled, y_train)
test_accuracy = model.score(X_test_scaled, y_test)

print(f"\nTraining Accuracy: {train_accuracy * 100:.2f}%")
print(f"Testing Accuracy:  {test_accuracy * 100:.2f}%")

# Step 8: Evaluation Metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Dataset shape: (980, 190)

Training Accuracy: 100.00%
Testing Accuracy:  100.00%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00        35
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00        41
           5       1.00      1.00      1.00        17
           6       1.00      1.00      1.00        50

    accuracy                           1.00       196
   macro avg       1.00      1.00      1.00       196
weighted avg       1.00      1.00      1.00       196

Confusion Matrix:
[[27  0  0  0  0  0  0]
 [ 0 11  0  0  0  0  0]
 [ 0  0 35  0  0  0  0]
 [ 0  0  0 15  0  0  0]
 [ 0  0  0  0 41  0  0]
 [ 0  0  0  0  0 17  0]
 [ 0  0  0  0  0  0 50]]


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load cleaned data
df = pd.read_csv("processed_faces_newFinal8_1/facial_features_and_landmarks_data_cleaned.csv")

# Encode target
le = LabelEncoder()
df["emotion_encoded"] = le.fit_transform(df["emotion"])

# Identify columns to drop (safely)
drop_cols = ["emotion", "filename", "emotion_encoded"]
drop_cols = [col for col in drop_cols if col in df.columns]

# Features and target
X = df.drop(columns=drop_cols)
y = df["emotion_encoded"]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predictions
y_pred_train = knn.predict(X_train)
y_pred_test = knn.predict(X_test)

# Accuracy scores
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)

# Results
print(f"Dataset shape: {df.shape}")
print(f"\nTraining Accuracy: {train_acc:.2%}")
print(f"Testing Accuracy:  {test_acc:.2%}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_test))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))


Dataset shape: (980, 191)

Training Accuracy: 91.71%
Testing Accuracy:  72.96%

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.59      0.54        27
           1       0.50      0.64      0.56        11
           2       0.65      0.63      0.64        35
           3       0.75      0.60      0.67        15
           4       0.86      0.90      0.88        41
           5       0.40      0.35      0.38        17
           6       1.00      0.92      0.96        50

    accuracy                           0.73       196
   macro avg       0.67      0.66      0.66       196
weighted avg       0.74      0.73      0.73       196

Confusion Matrix:
[[16  2  5  0  0  4  0]
 [ 1  7  0  0  0  3  0]
 [ 7  1 22  0  5  0  0]
 [ 2  2  0  9  1  1  0]
 [ 0  1  2  1 37  0  0]
 [ 6  1  4  0  0  6  0]
 [ 0  0  1  2  0  1 46]]


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load cleaned dataset
df = pd.read_csv("processed_faces_newFinal8_1/facial_features_and_landmarks_data_cleaned.csv")
print("Dataset shape:", df.shape)

# Encode emotion labels
le = LabelEncoder()
df["emotion_encoded"] = le.fit_transform(df["emotion"])

# Drop unnecessary columns (if any exist)
X = df.drop(columns=["emotion", "emotion_encoded"], errors='ignore')
y = df["emotion_encoded"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Evaluate
train_acc = dt_model.score(X_train, y_train)
test_acc = dt_model.score(X_test, y_test)

print(f"\nTraining Accuracy: {train_acc * 100:.2f}%")
print(f"Testing Accuracy:  {test_acc * 100:.2f}%")

# Detailed Report
y_pred = dt_model.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Dataset shape: (980, 190)

Training Accuracy: 100.00%
Testing Accuracy:  86.22%

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.74      0.72        23
           1       0.43      0.67      0.52         9
           2       0.95      0.83      0.89        42
           3       0.83      0.83      0.83        18
           4       0.93      0.95      0.94        42
           5       0.81      0.85      0.83        20
           6       1.00      0.93      0.96        42

    accuracy                           0.86       196
   macro avg       0.81      0.83      0.81       196
weighted avg       0.88      0.86      0.87       196

Confusion Matrix:
[[17  4  1  0  0  1  0]
 [ 1  6  0  0  0  2  0]
 [ 3  0 35  2  2  0  0]
 [ 0  2  0 15  1  0  0]
 [ 0  1  1  0 40  0  0]
 [ 2  1  0  0  0 17  0]
 [ 1  0  0  1  0  1 39]]


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load preprocessed dataset
df = pd.read_csv("processed_faces_newFinal8_1/facial_features_and_landmarks_data_cleaned.csv")

# Encode emotion labels
le = LabelEncoder()
df["emotion_encoded"] = le.fit_transform(df["emotion"])

# Prepare features and target
X = df.drop(columns=["emotion", "emotion_encoded"], errors='ignore')  # Drop if present
y = df["emotion_encoded"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

# Initialize Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Evaluation
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print("Dataset shape:", df.shape)
print(f"\nTraining Accuracy: {train_accuracy * 100:.2f}%")
print(f"Testing Accuracy:  {test_accuracy * 100:.2f}%")

# Detailed report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_test))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))


Dataset shape: (980, 191)

Training Accuracy: 100.00%
Testing Accuracy:  96.43%

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93        27
           1       1.00      1.00      1.00        11
           2       0.97      0.89      0.93        35
           3       1.00      1.00      1.00        15
           4       0.93      1.00      0.96        41
           5       1.00      0.88      0.94        17
           6       1.00      1.00      1.00        50

    accuracy                           0.96       196
   macro avg       0.97      0.96      0.97       196
weighted avg       0.97      0.96      0.96       196

Confusion Matrix:
[[26  0  1  0  0  0  0]
 [ 0 11  0  0  0  0  0]
 [ 1  0 31  0  3  0  0]
 [ 0  0  0 15  0  0  0]
 [ 0  0  0  0 41  0  0]
 [ 2  0  0  0  0 15  0]
 [ 0  0  0  0  0  0 50]]


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load the preprocessed dataset
df = pd.read_csv("processed_faces_newFinal8_1/facial_features_and_landmarks_data_cleaned.csv")

# Encode emotion labels
le = LabelEncoder()
df["emotion_encoded"] = le.fit_transform(df["emotion"])

# Split into features and labels
X = df.drop(columns=["emotion", "emotion_encoded"], errors="ignore")
y = df["emotion_encoded"]

# Split into training and testing sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# Standardize features (important for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train SVM model
svm_model = SVC(kernel="rbf", C=1.0, gamma="scale", random_state=42)
svm_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_train = svm_model.predict(X_train_scaled)
y_pred_test = svm_model.predict(X_test_scaled)

# Accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Results
print("Dataset shape:", df.shape)
print(f"\nTraining Accuracy: {train_accuracy * 100:.2f}%")
print(f"Testing Accuracy:  {test_accuracy * 100:.2f}%")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))


Dataset shape: (980, 191)

Training Accuracy: 98.60%
Testing Accuracy:  93.37%

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.85      0.85        27
           1       0.91      0.91      0.91        11
           2       0.91      0.89      0.90        35
           3       1.00      1.00      1.00        15
           4       0.93      1.00      0.96        41
           5       0.88      0.82      0.85        17
           6       1.00      0.98      0.99        50

    accuracy                           0.93       196
   macro avg       0.93      0.92      0.92       196
weighted avg       0.93      0.93      0.93       196

Confusion Matrix:
[[23  1  3  0  0  0  0]
 [ 0 10  0  0  0  1  0]
 [ 1  0 31  0  3  0  0]
 [ 0  0  0 15  0  0  0]
 [ 0  0  0  0 41  0  0]
 [ 3  0  0  0  0 14  0]
 [ 0  0  0  0  0  1 49]]


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load preprocessed data
df = pd.read_csv("processed_faces_newFinal8_1/facial_features_and_landmarks_data_cleaned.csv")

# Label encode the target
le = LabelEncoder()
df['emotion_encoded'] = le.fit_transform(df['emotion'])

# Features and target
X = df.drop(columns=['emotion', 'emotion_encoded'], errors='ignore')
y = df['emotion_encoded']
num_classes = len(le.classes_)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One-hot encoding for labels
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

# Define the model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train_scaled, y_train_cat, epochs=50, batch_size=32, verbose=0, validation_split=0.1)

# Predict
y_pred_train = np.argmax(model.predict(X_train_scaled), axis=1)
y_pred_test = np.argmax(model.predict(X_test_scaled), axis=1)

# Accuracy
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)

# Report
print("Dataset shape:", df.shape)
print(f"\nTraining Accuracy: {train_acc * 100:.2f}%")
print(f"Testing Accuracy:  {test_acc * 100:.2f}%")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))


KeyboardInterrupt: 